In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns=None
pd.options.mode.chained_assignment = None


C:\Users\User\AppData\Local\Temp\ipykernel_16744\460341617.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
actions = pd.read_csv(r'C:\Users\User\apr_test\actions.csv')
triggers = pd.read_csv(r'C:\Users\User\triggers\triggers.csv')

In [3]:
triggers.head()

,guid,date,trigger,type
0,018bdd75-83fc-2510-50e7-153f7b5e4b64,2024-05-31 23:55:27,2047,2
1,0187a472-becb-7994-5e1e-f179b60bc141,2024-05-31 23:55:27,168,2
2,018ba175-9e02-8c7a-e522-6e5fec6949a0,2024-05-31 23:55:26,1807,2
3,018ba175-9e02-8c7a-e522-6e5fec6949a0,2024-05-31 23:55:26,2048,2
4,018e9cfb-c8bc-f38b-e209-5a77c5324a14,2024-05-31 23:55:26,1773,2


In [4]:
# объединение данных по guid, чтобы сопоставить действия с триггерами одного пользователя
merged_data = pd.merge(triggers, actions[['guid', 'date', 'result']], on='guid', how='inner')

# фильтрация триггеров, которые произошли до действия
merged_data = merged_data[merged_data['date_x'] <= merged_data['date_y']]


# Сортировка по guid и дате действия
merged_data = merged_data.sort_values(by=['guid', 'date_y', 'date_x'])

In [5]:

# Для каждого действия выбираем последний триггер до действия (максимальную дату триггера)
closest_trigger = merged_data.groupby(['guid', 'date_y']).tail(1).reset_index(drop=True)
# Переименуем столбцы для удобства
closest_trigger = closest_trigger.rename(columns={'date_x': 'date', 'date_y': 'action_date'})

In [6]:
triggers_actions = pd.merge(triggers,closest_trigger,how='left',on = ['guid','date','trigger','type'])

In [7]:
triggers_actions.shape

(43074627, 6)

In [8]:
triggers_actions.head()

,guid,date,trigger,type,action_date,result
0,018bdd75-83fc-2510-50e7-153f7b5e4b64,2024-05-31 23:55:27,2047,2,NaN,NaN
1,0187a472-becb-7994-5e1e-f179b60bc141,2024-05-31 23:55:27,168,2,NaN,NaN
2,018ba175-9e02-8c7a-e522-6e5fec6949a0,2024-05-31 23:55:26,1807,2,NaN,NaN
3,018ba175-9e02-8c7a-e522-6e5fec6949a0,2024-05-31 23:55:26,2048,2,NaN,NaN
4,018e9cfb-c8bc-f38b-e209-5a77c5324a14,2024-05-31 23:55:26,1773,2,2024-05-31 23:57:14,0.0


In [9]:
triggers_actions.to_csv(r'C:\Users\User\triggers\triggers_actions.csv',index = False)